In [2]:
import pandas as pd
import numpy as np
# import scattertext as st
# import spacy
import time
from IPython.display import IFrame
from IPython.core.display import display, HTML
from bokeh.palettes import PuBu
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource, ranges, LabelSet
from bokeh.plotting import figure
output_notebook()
display(HTML("<style>.container { width:50% !important; }</style>"))
%matplotlib inline
from tqdm import tqdm
import openreview

Loading BokehJS ...

In [3]:
url = 'https://api.openreview.net/notes?invitation=ICLR.cc%2F2018%2FConference%2F-%2FBlind_Submission&offset=0&limit=1000'
df = pd.DataFrame(requests.get(url).json()['notes'])

In [4]:
forum_content = []
for i, forum_id in tqdm(list(enumerate(df.forum))):
    notes_url = 'https://api.openreview.net/notes?forum={}&trash=true'.format(forum_id)
    try:
        forum_content.append(requests.get(notes_url).json())
    except:
        print('err', i, forum_id)
        forum_content = {}
    time.sleep(.3)
df['forumContent'] = pd.Series(forum_content)

100%|████████████████████████████████████████████████████████████████████████████████| 911/911 [07:11<00:00,  2.11it/s]


In [5]:
df['abstract'] = df['content'].apply(lambda x: x.get('abstract'))

In [8]:
df['decision_raw'] = df.forumContent.apply(lambda x:[n['content']['decision'] 
                                                     for n in x['notes'] 
                                                     if 'decision' in n['content']][0])
df['decision_raw'].value_counts()

Reject                      486
Accept (Poster)             313
Invite to Workshop Track     89
Accept (Oral)                23
Name: decision_raw, dtype: int64

In [9]:
df['title'] = df.content.apply(lambda x: x['title'])
df['authors'] = df.content.apply(lambda x: x['authors'])

only_reviews_df = pd.concat(df.forumContent.apply(lambda c: pd.DataFrame([
    {'review': n['content']['review'], 
     'rating': n['content']['rating'],  
     'confidence': n['content']['confidence'],
     'forum': n['forum']} 
    for n in c['notes'] 
    if 'content' in n and 'review' in n['content']
])).tolist())
reviews_df = pd.merge(df[['title', 'authors', 'decision_raw', 'forum', 'abstract']], only_reviews_df, on='forum')
#reviews_df.groupby('decision_raw')['rating'].value_counts()
reviews_df['decision'] = (reviews_df['decision_raw']
                          .apply(lambda x: 'Reject' if x == 'Reject' 
                                 else ('Accept' if x.startswith('Accept') 
                                       else 'Workshop')))
reviews_df['rating_bin'] = (reviews_df['rating']
                            .apply(lambda x: (lambda s: 'Negative' if s < 5 
                                              else ('Positive' if s > 6 else 'Neutral'))
                                   (int(x.split(':')[0].strip()))))
reviews_df['category'] = reviews_df['decision'] + ', ' + reviews_df['rating_bin']

In [10]:
reviews_df['metadata'] = (
    reviews_df['title'] + '<br/>Score: ' + reviews_df['rating'].apply(lambda x: x.split(':')[0]) + '/10'
    + '<br/>Confidence: ' + reviews_df['confidence'].apply(lambda x: x.split(':')[0]) + '/5'
    + '<br/>Ultimate decision: ' + reviews_df['decision'].apply(lambda x: x.split(':')[0]) + '/10'
)

In [11]:
reviews_df['title'].nunique()

910

In [ ]:
reviews_df.to_csv('peer_reviews.csv', index = False)